In [1]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Book Assistant v2",
    instructions="You help users with their questions on the files they upload as well as the files that were already uploaded via openai assistant tool.",
    tools=[{"type": "file_search"}],
    model="gpt-4o",
)

# assistant_id = "asst_R0iQNbINw8zNsXGeppTnBZYD"

In [2]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want you to answer my questions with the files I uplaoded. When you answer my question, you MUST include the source citations with the full source text at the bottom so I can easily find the sources.",
)

thread

Thread(id='thread_bpYRvyh1beoTVXv30pyMA5My', created_at=1720487956, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [8]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

print(run)

Run(id='run_7w7jSC2x7l5GKeJJAdByCgJs', assistant_id='asst_flpe01vTZ9AwYkvzx4o3jjZw', cancelled_at=None, completed_at=None, created_at=1720487982, expires_at=1720488582, failed_at=None, incomplete_details=None, instructions='You help users with their questions on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_bpYRvyh1beoTVXv30pyMA5My', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [4]:
with open("./files/Orwell_1984.pdf", "rb") as file_object:
    file = client.files.create(
        file=file_object,
        purpose="assistants",
    )

file

FileObject(id='file-hgHUUfojznABoFkjd7cAqXpt', bytes=1644307, created_at=1720487962, filename='Orwell_1984.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [5]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please refer to the uploaded file for my question. ",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {
                    "type": "file_search",
                }
            ],
        }
    ],
)

Message(id='msg_D9cARfloZfRDeOVK6lZ6FRh2', assistant_id=None, attachments=[Attachment(file_id='file-hgHUUfojznABoFkjd7cAqXpt', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question. '), type='text')], created_at=1720487966, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_bpYRvyh1beoTVXv30pyMA5My')

In [6]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

In [12]:
get_messages(thread.id)

user: I want you to answer my questions with the files I uplaoded. When you answer my question, you MUST include the source citations with the full source text at the bottom so I can easily find the sources.
assistant: Absolutely, I'll make sure to include citations with full source text at the bottom of my responses. Please go ahead and upload your files, and let me know your specific questions.
user: Please refer to the uploaded file for my question. 
user: I want to know where Winston lives and how his residence looks like.
assistant: Winston Smith lives in Victory Mansions, a large apartment block. His flat is located seven flights up, and he has to take the stairs since the lift rarely works due to frequent power cuts implemented as part of an economy drive. The hallway of the building smells of "boiled cabbage and old rag mats," depicting a rather bleak and decayed atmosphere. Inside his flat, there is a telescreen that broadcasts and monitors simultaneously, making it impossible

In [11]:
get_run(run.id, thread.id).status

'completed'

In [ ]:
get_messages(thread.id)

In [7]:
send_message(
    thread.id, "I want to know where Winston lives and how his residence looks like."
)

Message(id='msg_9jOK4beTnh7znypMbf47jnpd', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know where Winston lives and how his residence looks like.'), type='text')], created_at=1720487977, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_bpYRvyh1beoTVXv30pyMA5My')

In [ ]:
get_messages(thread.id)